# Statewide Data with Injury Type as Target

## Imports and Setup 

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import psycopg2 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Load in password
from secret import secret, database, username, host

# Establish connection to database
conn = psycopg2.connect(
    host = host,
    database = database,
    user =username,
    password = secret
)

In [3]:
# Load the data
query = 'SELECT * FROM clark_co_traffic'
traffic_df = pd.read_sql_query(query, conn)

/var/folders/f3/5zcd9_1j7d3546ws7g9t5fw00000gp/T/ipykernel_6046/3027837082.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  traffic_df = pd.read_sql_query(query, conn)


In [4]:
# Drop unnecessary columns
traffic_df.drop(columns = ['objectid','crash_severity','accident_rec_num','primary_street', 'secondary_street', 'fatalities', 'injured' ], inplace = True)

In [5]:
# Move PDO values from property_damage_only column info to injury_type column
traffic_df['injury_type'] = np.where(traffic_df['property_damage_only'] == 'PDO', traffic_df['property_damage_only'], traffic_df['injury_type'])

In [6]:
# Drop property damage only column
traffic_df.drop(columns = ['property_damage_only'], inplace = True)

In [7]:
# Split single column with multiple data points into separate columns
traffic_df[['v1_driver_factors_1', 'v1_driver_factors_2','v1_driver_factors_3', 'v1_driver_factors_4']] = traffic_df['v1_driver_factors'].str.split(':', expand=True)

traffic_df[['v1_vehicle_factors_1', 'v1_vehicle_factors_2','v1_vehicle_factors_3', 'v1_vehicle_factors_4', 'v1_vehicle_factors_5']] = traffic_df['v1_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v1_all_events_1', 'v1_all_events_2','v1_all_events_3', 'v1_all_events_4', 'v1_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['v2_driver_factors_1', 'v2_driver_factors_2']] = traffic_df['v2_driver_factors'].str.split(':', expand=True)

traffic_df[['v2_vehicle_factors_1', 'v2_vehicle_factors_2','v2_vehicle_factors_3', 'v2_vehicle_factors_4', 'v2_vehicle_factors_5']] = traffic_df['v2_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v2_all_events_1', 'v2_all_events_2','v2_all_events_3', 'v2_all_events_4', 'v2_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['nonmotorist_factors_1', 'nonmotorist_factors_2','nonmotorist_factors_3', 'nonmotorist_factors_4', 'nonmotorist_factors_5']] = traffic_df['nonmotorist_factors'].str.split(':', expand=True)

traffic_df[['factors_roadway_1', 'factors_roadway_2','factors_roadways_3']] = traffic_df['factors_roadway'].str.split(':', expand=True)

traffic_df[['hwy_factors_1', 'hwy_factors_2','hwy_factors_3', 'hwy_factors_4']] = traffic_df['hwy_factors'].str.split(':', expand=True)

traffic_df.drop(columns = ['v1_driver_factors','v1_vehicle_factors','v1_all_events','v2_driver_factors','v2_vehicle_factors', 'v2_all_events', 'nonmotorist_factors', 'factors_roadway', 'hwy_factors',], inplace = True)

In [8]:
traffic_df['injury_type'].value_counts()

PDO    128261
C       69561
B       22594
A        3836
K        1175
U         241
Name: injury_type, dtype: int64

In [9]:
# Change target values to numeric
# Values are based on Nevada Traffic Records Coordinating Committee Data Dictionary,
# https://zerofatalitiesnv.com/app/uploads/2021/04/2021-01-NV-TRCC-Data-Dictionary.pdf
injury_num = {
    'K': 5,
    'A': 4,
    'B': 3,
    'C': 2,
    'PDO': 1,
    'U': 0,    
}

traffic_df['injury_type'] = traffic_df['injury_type'].apply(lambda x: injury_num[x])

traffic_df['injury_type'].value_counts()

1    128261
2     69561
3     22594
4      3836
5      1175
0       241
Name: injury_type, dtype: int64

In [10]:
# Extract datetime values into separate columns
traffic_df['crash_month'] = pd.DatetimeIndex(traffic_df['crash_date']).month
traffic_df['crash_day'] = pd.DatetimeIndex(traffic_df['crash_date']).day
traffic_df['crash_day_of_week'] = pd.DatetimeIndex(traffic_df['crash_date']).dayofweek
traffic_df['crash_hour'] = pd.DatetimeIndex(traffic_df['crash_time']).hour


In [11]:
# Drop original datetime columns
traffic_df.drop(columns = ['crash_date', 'crash_time'], inplace = True)

In [12]:
traffic_df.dtypes

x                    float64
y                    float64
county                object
crash_year             int64
weather               object
                      ...   
hwy_factors_4         object
crash_month            int64
crash_day              int64
crash_day_of_week      int64
crash_hour             int64
Length: 65, dtype: object

In [13]:
# Get names of all columns with dtypes of "object"
sel_cols = list(traffic_df.select_dtypes(include='object'))


In [14]:
# Preprocessing to convert all strings to numeric values
traffic_df_encoded = pd.get_dummies(traffic_df, columns = sel_cols)
traffic_df_encoded.head()

,x,y,crash_year,injury_type,total_vehicles,v1_driver_age,v2_driver_age,crash_month,crash_day,crash_day_of_week,...,hwy_factors_3_ OTHER HIGHWAY,hwy_factors_3_ ROAD OBSTRUCTION,"hwy_factors_3_ RUTS, HOLES, BUMPS",hwy_factors_3_ SHOULDERS,hwy_factors_3_ VISUAL OBSTRUCTION(S),hwy_factors_3_ WEATHER,"hwy_factors_3_ WET, ICY, SNOW, SLUSH",hwy_factors_3_ WORK ZONE (CONST. MAINT. UTILITY),hwy_factors_3_ WORN TRAFFIC SURFACE,"hwy_factors_4_ WET, ICY, SNOW, SLUSH"
0,-115.106709,36.236043,2016,1,2,35,41,1,4,0,...,0,0,0,0,0,0,0,0,0,0
1,-119.673794,39.626433,2016,1,1,39,41,1,7,3,...,0,0,0,0,0,0,0,0,0,0
2,-115.101063,36.240200,2016,1,2,31,53,1,4,0,...,0,0,0,0,0,0,0,0,0,0
3,-119.633092,39.516952,2016,1,1,18,41,1,6,2,...,0,0,0,0,0,0,0,0,0,0
4,-115.140570,36.192680,2016,1,1,56,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0


## Model with Injury Type Crash Severity as Target and All Other Columns as Features

In [15]:
# Separate data into features and target
y = traffic_df_encoded['injury_type'].values
X = traffic_df_encoded.drop(columns = 'injury_type')

In [16]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)
X_train.shape

(169251, 1345)

In [17]:
# Scale the model
scaler = StandardScaler() 

# Fit the scaler with the training data
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Create and fit the Random Forest model
rf_model = RandomForestClassifier(n_estimators = 128)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [19]:
# Create predictions
predictions = rf_model.predict(X_test_scaled)
results = pd.DataFrame({
    "Prediction": predictions,
    "Actual": y_test
}).reset_index(drop = True)
results.head()

,Prediction,Actual
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [20]:
# Assess accuracy score
accuracy = accuracy_score(y_test, predictions)
accuracy

0.6552103089494301

In [21]:
# Generate confusion matrix and dataframe
cm = confusion_matrix(y_test, predictions)

index_values = ['Actual: Unknown', 
                'Actual: Property Damage Only', 
                'Actual: Possible Injury', 
                'Actual: Suspected Minor Injury', 
                'Actual: Suspected Serious Injury', 
                'Actual: Fatal Injury']

column_values = ['Predicted: Unknown', 
                'Predicted: Property Damage Only', 
                'Predicted: Possible Injury', 
                'Predicted: Suspected Minor Injury', 
                'Predicted: Suspected Serious Injury', 
                'Predicted: Fatal Injury']

cm_df = pd.DataFrame(cm, index = index_values, columns = column_values)
cm_df
                  

,Predicted: Unknown,Predicted: Property Damage Only,Predicted: Possible Injury,Predicted: Suspected Minor Injury,Predicted: Suspected Serious Injury,Predicted: Fatal Injury
Actual: Unknown,0,31,21,8,0,0
Actual: Property Damage Only,0,27226,4718,111,8,2
Actual: Possible Injury,0,7846,9051,482,10,1
Actual: Suspected Minor Injury,1,2275,2722,620,30,1
Actual: Suspected Serious Injury,0,324,382,213,33,7
Actual: Fatal Injury,0,118,67,55,19,35


## Results with Injury Type as Target and All Other Columns as Features

In [22]:
# Display results
print('Confusion Matrix')
display(cm_df)
print(f'\nAccuracy Score: {accuracy}\n')
print('Classification Report')
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted: Unknown,Predicted: Property Damage Only,Predicted: Possible Injury,Predicted: Suspected Minor Injury,Predicted: Suspected Serious Injury,Predicted: Fatal Injury
Actual: Unknown,0,31,21,8,0,0
Actual: Property Damage Only,0,27226,4718,111,8,2
Actual: Possible Injury,0,7846,9051,482,10,1
Actual: Suspected Minor Injury,1,2275,2722,620,30,1
Actual: Suspected Serious Injury,0,324,382,213,33,7
Actual: Fatal Injury,0,118,67,55,19,35



Accuracy Score: 0.6552103089494301

Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.72      0.85      0.78     32065
           2       0.53      0.52      0.53     17390
           3       0.42      0.11      0.17      5649
           4       0.33      0.03      0.06       959
           5       0.76      0.12      0.21       294

    accuracy                           0.66     56417
   macro avg       0.46      0.27      0.29     56417
weighted avg       0.62      0.66      0.62     56417



### Importances with Injury Type as Target and All Other Columns as Features

In [23]:
# Calculate importance of features
importances = rf_model.feature_importances_
important = sorted(zip(importances, X.columns), reverse = True)
important

[(0.05334433024803446, 'y'),
 (0.05275042600479136, 'x'),
 (0.052305265504493985, 'v1_driver_age'),
 (0.045160578019586874, 'crash_day'),
 (0.04481583187156313, 'v2_driver_age'),
 (0.04198900670515177, 'crash_hour'),
 (0.03851475257970124, 'crash_month'),
 (0.03421103512319021, 'crash_day_of_week'),
 (0.022825270294973027, 'crash_year'),
 (0.019535585098099648, 'lighting_UNKNOWN'),
 (0.01582573508335977, 'factors_roadway_1_UNKNOWN'),
 (0.015205762099211017, 'factors_roadway_1_DRY'),
 (0.013555085229155488, 'hwy_factors_1_NONE'),
 (0.0129464616401986, 'hwy_factors_1_UNKNOWN'),
 (0.012404732183429728, 'total_vehicles'),
 (0.011606840975046794, 'lighting_DAYLIGHT'),
 (0.011460636499868527, 'v1_type_SEDAN, 4 DOOR'),
 (0.008863885675325202, 'v1_type_CARRY-ALL'),
 (0.00855236532754442, 'v1_action_GOING STRAIGHT'),
 (0.008215598969628353, 'v1_driver_factors_1_APPARENTLY NORMAL'),
 (0.008025597240316092, 'v2_type_SEDAN, 4 DOOR'),
 (0.007938755952329714, 'v1_vehicle_factors_1_FAILED TO YIELD RI

## New Version of Model Based on Top Importances

### Importances to exclude

In [24]:
low_priority = important[772:]


In [25]:
imp_only_list = []
listy_list = []

# iterate using index with enumerate function
for index, tuple in enumerate(low_priority):
       
    # access through index by appending to list
    imp_only_list.append(low_priority[index])
     
# iterate through the list
for x in imp_only_list:
    for y in x:
         listy_list.append(y)

final_list = listy_list[1::2]



### Statewide Dataframe without Low-Value Importances

In [26]:
important_df = traffic_df_encoded.drop(columns = final_list)
important_df.head()

,x,y,crash_year,injury_type,total_vehicles,v1_driver_age,v2_driver_age,crash_month,crash_day,crash_day_of_week,...,hwy_factors_2_ BACKUP DUE TO PRIOR NON-RECURRING INCIDENT,hwy_factors_2_ BACKUP DUE TO REGULAR CONGESTION,hwy_factors_2_ NON-ROADWAY WORK,hwy_factors_2_ OTHER HIGHWAY,"hwy_factors_2_ RUTS, HOLES, BUMPS","hwy_factors_2_ WET, ICY, SNOW, SLUSH",hwy_factors_2_ WORK ZONE (CONST. MAINT. UTILITY),hwy_factors_3_ BACKUP DUE TO PRIOR CRASH,hwy_factors_3_ OTHER HIGHWAY,hwy_factors_3_ WORK ZONE (CONST. MAINT. UTILITY)
0,-115.106709,36.236043,2016,1,2,35,41,1,4,0,...,0,0,0,0,0,0,0,0,0,0
1,-119.673794,39.626433,2016,1,1,39,41,1,7,3,...,0,0,0,0,0,0,0,0,0,0
2,-115.101063,36.240200,2016,1,2,31,53,1,4,0,...,0,0,0,0,0,0,0,0,0,0
3,-119.633092,39.516952,2016,1,1,18,41,1,6,2,...,0,0,0,0,0,0,0,0,0,0
4,-115.140570,36.192680,2016,1,1,56,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0


## Model with Injury Type as Target and Only High-Value Importances as Features

In [27]:
# Separate data into features and target
y_imp = important_df['injury_type'].values
X_imp = important_df.drop(columns = 'injury_type')

In [28]:
X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(X_imp, y_imp)
X_train_imp.shape

(169251, 772)

In [29]:
# Scale the model
scaler_imp = StandardScaler() 

# Fit the scaler with the training data
X_scaler_imp = scaler_imp.fit(X_train_imp)
X_train_scaled_imp = X_scaler_imp.transform(X_train_imp)
X_test_scaled_imp = X_scaler_imp.transform(X_test_imp)

In [30]:
# Create and fit the Random Forest model
rf_model_imp = RandomForestClassifier(n_estimators = 128)
rf_model_imp = rf_model.fit(X_train_scaled_imp, y_train_imp)

In [31]:
# Create predictions
predictions_imp = rf_model_imp.predict(X_test_scaled_imp)
results_imp = pd.DataFrame({
    "Prediction": predictions_imp,
    "Actual": y_test_imp
}).reset_index(drop = True)
results_imp.head()

,Prediction,Actual
0,1,1
1,2,3
2,2,1
3,2,2
4,1,1


In [32]:
# Assess accuracy score
accuracy_imp = accuracy_score(y_test_imp, predictions_imp)
accuracy_imp

0.6564510697130298

In [33]:
# Generate confusion matrix and dataframe
cm_imp = confusion_matrix(y_test_imp, predictions_imp)

index_values_imp = ['Actual: Unknown', 
                'Actual: Property Damage Only', 
                'Actual: Possible Injury', 
                'Actual: Suspected Minor Injury', 
                'Actual: Suspected Serious Injury', 
                'Actual: Fatal Injury']

column_values_imp = ['Predicted: Unknown', 
                'Predicted: Property Damage Only', 
                'Predicted: Possible Injury', 
                'Predicted: Suspected Minor Injury', 
                'Predicted: Suspected Serious Injury', 
                'Predicted: Fatal Injury']

cm_df_imp = pd.DataFrame(cm_imp, index = index_values_imp, columns = column_values_imp)
cm_df_imp

,Predicted: Unknown,Predicted: Property Damage Only,Predicted: Possible Injury,Predicted: Suspected Minor Injury,Predicted: Suspected Serious Injury,Predicted: Fatal Injury
Actual: Unknown,0,31,12,10,0,2
Actual: Property Damage Only,0,27140,4876,130,10,1
Actual: Possible Injury,0,7658,9195,440,27,1
Actual: Suspected Minor Injury,0,2190,2750,635,39,4
Actual: Suspected Serious Injury,0,343,372,218,25,11
Actual: Fatal Injury,0,130,48,60,19,40


## Statewide Results with Injury Type as Target and High-Priority Columns as Features

In [34]:
# Display results
print('Confusion Matrix')
display(cm_df_imp)
print(f'\nAccuracy Score: {accuracy_imp}\n')
print('Classification Report')
print(classification_report(y_test_imp, predictions_imp))

Confusion Matrix


,Predicted: Unknown,Predicted: Property Damage Only,Predicted: Possible Injury,Predicted: Suspected Minor Injury,Predicted: Suspected Serious Injury,Predicted: Fatal Injury
Actual: Unknown,0,31,12,10,0,2
Actual: Property Damage Only,0,27140,4876,130,10,1
Actual: Possible Injury,0,7658,9195,440,27,1
Actual: Suspected Minor Injury,0,2190,2750,635,39,4
Actual: Suspected Serious Injury,0,343,372,218,25,11
Actual: Fatal Injury,0,130,48,60,19,40



Accuracy Score: 0.6564510697130298

Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        55
           1       0.72      0.84      0.78     32157
           2       0.53      0.53      0.53     17321
           3       0.43      0.11      0.18      5618
           4       0.21      0.03      0.05       969
           5       0.68      0.13      0.22       297

    accuracy                           0.66     56417
   macro avg       0.43      0.27      0.29     56417
weighted avg       0.63      0.66      0.63     56417



/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Calculate importance of features

In [35]:

importances_high_priority = rf_model_imp.feature_importances_
sorted(zip(importances_high_priority, X_imp.columns), reverse = True)

[(0.05500245724608254, 'y'),
 (0.054390065770816974, 'x'),
 (0.05294854333063855, 'v1_driver_age'),
 (0.04524157120425445, 'v2_driver_age'),
 (0.04520645612241052, 'crash_day'),
 (0.04231583150648051, 'crash_hour'),
 (0.03841735428739241, 'crash_month'),
 (0.03385610318648117, 'crash_day_of_week'),
 (0.02241621565902101, 'crash_year'),
 (0.016312240238785296, 'factors_roadway_1_DRY'),
 (0.01580605601490648, 'lighting_UNKNOWN'),
 (0.015486594512447747, 'factors_roadway_1_UNKNOWN'),
 (0.015373089751500273, 'hwy_factors_1_NONE'),
 (0.012840290440794183, 'total_vehicles'),
 (0.012364924746681724, 'hwy_factors_1_UNKNOWN'),
 (0.01225154538439265, 'lighting_DAYLIGHT'),
 (0.011254160968122158, 'v1_type_SEDAN, 4 DOOR'),
 (0.00891042971411906, 'v1_type_CARRY-ALL'),
 (0.008676875895753859, 'v2_type_SEDAN, 4 DOOR'),
 (0.008434978484779152, 'v1_action_GOING STRAIGHT'),
 (0.008173240847406215, 'v1_driver_factors_1_APPARENTLY NORMAL'),
 (0.008007981047573953, 'v1_vehicle_factors_1_FAILED TO YIELD RIG